In [1]:
!python -V

Python 3.10.13


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [28]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-12.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')

In [7]:
len(df_train), len(df_val)

(59603, 66097)

In [29]:
df_train

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,1,2023-02-01 00:46:22,2023-02-01 01:05:57,N,1.0,74,265,1.0,10.80,42.90,...,1.5,0.00,0.0,NaN,1.0,45.40,2.0,1.0,0.00,19.583333
1,2,2023-02-01 00:05:09,2023-02-01 00:22:42,N,1.0,216,196,1.0,4.76,23.30,...,0.5,0.00,0.0,NaN,1.0,25.80,2.0,1.0,0.00,17.550000
2,2,2023-02-01 00:03:47,2023-02-01 00:27:30,N,1.0,7,114,1.0,6.32,30.30,...,0.5,8.89,0.0,NaN,1.0,44.44,1.0,1.0,2.75,23.716667
3,2,2023-01-31 23:30:56,2023-01-31 23:51:40,N,1.0,74,239,1.0,3.50,16.30,...,0.5,0.00,0.0,NaN,1.0,21.55,2.0,1.0,2.75,20.733333
4,2,2023-02-01 00:15:05,2023-02-01 00:26:02,N,1.0,82,223,1.0,3.14,17.00,...,0.5,0.00,0.0,NaN,1.0,19.50,2.0,1.0,0.00,10.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64804,2,2023-02-28 21:49:00,2023-02-28 22:04:00,None,NaN,7,263,NaN,3.83,16.70,...,0.0,2.00,0.0,NaN,1.0,22.45,NaN,NaN,NaN,15.000000
64805,2,2023-02-28 21:05:00,2023-02-28 21:08:00,None,NaN,41,42,NaN,0.67,12.95,...,0.0,2.79,0.0,NaN,1.0,16.74,NaN,NaN,NaN,3.000000
64806,2,2023-02-28 22:42:00,2023-02-28 22:59:00,None,NaN,166,141,NaN,3.91,19.60,...,0.0,3.50,0.0,NaN,1.0,26.85,NaN,NaN,NaN,17.000000
64807,2,2023-02-28 23:21:00,2023-02-28 23:38:00,None,NaN,41,244,NaN,4.54,19.36,...,0.0,4.07,0.0,NaN,1.0,24.43,NaN,NaN,NaN,17.000000


In [11]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
val_dicts[0]

{'PULocationID': '43', 'DOLocationID': '238', 'trip_distance': 1.16}

In [17]:
X_train[0]

<1x471 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

8.19383255249626

In [21]:
from sklearn.pipeline import make_pipeline

In [22]:
pipeline = make_pipeline(
    DictVectorizer(),
    LinearRegression()
)

In [23]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')


In [24]:
pipeline.fit(train_dicts, y_train)

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('linearregression', LinearRegression())])

In [26]:
y_pred = pipeline.predict(val_dicts)

mean_squared_error(y_val, y_pred, squared=False)

8.19383255249626

In [27]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)